In [2]:
import pandas as pd
from python_secrets import *
import requests


In [2]:
df = pd.read_csv('incident2.csv')
df.head()

,block_number,user_tx,fees
0,19412019,0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8...,1.861322e+16
1,19412030,0x354e8386267ca643793de913739df3f9680895776f3a...,4.694753e+16
2,19412041,0xee8fd2c76181afa14ca0da158e0a01bba2d3df8e62c5...,6.182506e+16
3,19412043,0x5f3954cc3cb4fbb88803a910d852aab6566af2866acf...,1.283444e+17
4,19412043,0xbd9c86df3327871d630a0921c65cb7bec9b0c199d80c...,3.640861e+16


# use this api to find the position in block from transaction hash
# https://docs.infura.io/api/networks/ethereum/json-rpc-methods/eth_gettransactionreceipt


example: for the first transaction from the file we have, it has the transaction hash: 0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581

By looking at etherscan: 
- the user swapped  849 BRETT for 0.4 ETH on Uniswap v2
- (from block 19412019)
- a contract address: 0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D

By looking at zeromev:
- the transaction was on position 2 (3d transaction)

In [26]:
# checking for that one transaction in infura

import requests
import json
import python_secrets

url = f"https://mainnet.infura.io/v3/{infura_api_key}"

payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "eth_getTransactionReceipt",
  "params": [
    "0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581"
  ],
  "id": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"jsonrpc":"2.0","id":1,"result":{"blockHash":"0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a2364ffa8cab9910143ee01","blockNumber":"0x1283433","contractAddress":null,"cumulativeGasUsed":"0x6f884","effectiveGasPrice":"0x11cd060113","from":"0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179","gasUsed":"0x2038c","logs":[{"address":"0xb0699d63aef20df3f1cffa9ca2bb8670416271d2","blockHash":"0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a2364ffa8cab9910143ee01","blockNumber":"0x1283433","data":"0x00000000000000000000000000000000000000000000002e092d563911a14d34","logIndex":"0xd","removed":false,"topics":["0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef","0x00000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed36179","0x000000000000000000000000e1ebdf64f7f3a31723e767a561345f958233bb7d"],"transactionHash":"0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581","transactionIndex":"0x2"},{"address":"0xb0699d63aef20df3f1cffa9ca2bb8670416271d2","blockHash":"0x2c1ca96b35f726a21

In [13]:
pd.DataFrame.from_dict(response.json()['result'], orient='index')


,0
blockHash,0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a23...
blockNumber,0x1283433
contractAddress,None
cumulativeGasUsed,0x6f884
effectiveGasPrice,0x11cd060113
from,0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179
gasUsed,0x2038c
logs,[{'address': '0xb0699d63aef20df3f1cffa9ca2bb86...
logsBloom,0x00200000000000000000000080000000000000000000...
status,0x1


In [15]:
response.json()['result']['blockHash']

'0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a2364ffa8cab9910143ee01'

In [5]:
# we see that it was on position 0x2 hexa- decimal which is position 2 (third transaction)

# Get the unix timestamp of the transaction

Our transaction is  at time stamp Mar-11-2024 01:00:59 PM +UTC


In [21]:
url = "https://api.etherscan.io/api?module=account&action=txlistinternal&txhash=0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581&apikey=A3UNV5BPTMEXUUSYES5VY12KE2FBQ42HQ6"

response = requests.request("POST", url)
print(response.text)

{"status":"1","message":"OK","result":[{"blockNumber":"19412019","timeStamp":"1710162059","from":"0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2","to":"0x7a250d5630b4cf539739df2c5dacb4c659f2488d","value":"400000000000000000","contractAddress":"","input":"","type":"call","gas":"2300","gasUsed":"83","isError":"0","errCode":""},{"blockNumber":"19412019","timeStamp":"1710162059","from":"0x7a250d5630b4cf539739df2c5dacb4c659f2488d","to":"0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179","value":"400000000000000000","contractAddress":"","input":"","type":"call","gas":"45427","gasUsed":"0","isError":"0","errCode":""}]}


idea instead of getting the timstamp of the transaction, get the timestamp of the block, so to avoid doing repetitive API calls in case we alreay have the block in our dataset. Use this endoint 


In [27]:
url = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno=19412019&apikey={eth_scan_api_key}"

response2 = requests.request("POST", url)
print(response2.text)   

NameError: name 'eth_scan_api_key' is not defined

# simulate the transaction if it was in the first block position

using phalcon blocksec we simulate if it was in position 0 in the block:
- block 19412019
- swap 807 BRETT for 0.4 ETH (instead of 849 if in position 3)

In [192]:
headers = {
    'X-Access-Key': f'{tenderly_access_token}',
    'content-type': 'application/json',
}

json_data = {
    'network_id': '1',
    'from': '0x36A2fFB33C1B427C46C3D30Adac3cA4e8ED36179',
    'to': '0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D',
    'input': '0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',

    'block_number': 19412019,
    'transaction_index': 0,
    #'value': 0,
    'simulation_type': 'quick'

}

response = requests.post(
    'https://api.tenderly.co/api/v1/account/aurelie/project/cowswap/simulate',
    headers=headers,
    json=json_data,
)

In [193]:
print(response.json())

with open("Output.json", "w") as text_file:
    text_file.write(response.text)

{'transaction': {'hash': '0xd9488fe37e24c1b3f14937e3029036e6aa480d1e1621d81a96ef0779c492f64b', 'block_hash': '', 'block_number': 19412019, 'from': '0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179', 'gas': 9223372036854775807, 'gas_price': 0, 'gas_fee_cap': 0, 'gas_tip_cap': 0, 'cumulative_gas_used': 0, 'gas_used': 142396, 'effective_gas_price': 0, 'input': '0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'nonce': 1799, 'to': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d', 'index': 0, 'value': '0x', 'access_list': None, 'stat

In [121]:
response.json()

{'transaction': {'hash': '0xd9488fe37e24c1b3f14937e3029036e6aa480d1e1621d81a96ef0779c492f64b',
  'block_hash': '',
  'block_number': 19412019,
  'from': '0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179',
  'gas': 9223372036854775807,
  'gas_price': 0,
  'gas_fee_cap': 0,
  'gas_tip_cap': 0,
  'cumulative_gas_used': 0,
  'gas_used': 142396,
  'effective_gas_price': 0,
  'input': '0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
  'nonce': 1799,
  'to': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d',
  'index': 0,
  'value': '0

In [141]:
for i in range(5):
    print(response.json()['transaction']['transaction_info']['asset_changes'][i]['raw_amount'])
    print(response.json()['transaction']['transaction_info']['asset_changes'][i]['dollar_value'])

807452531860501402532
None
400000000000000000
1431.5080078125
400000000000000000
1431.5080078125
400000000000000000
1432.60400390625
400000000000000000
1432.60400390625


In [133]:
response.json()['transaction']['transaction_info']['balance_changes']

[{'address': '0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179',
  'dollar_value': '1432.60400390625',
  'transfers': [0, 4]},
 {'address': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d',
  'dollar_value': '0',
  'transfers': [1, 2, 3, 4]},
 {'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
  'dollar_value': '-1432.60400390625',
  'transfers': [3]},
 {'address': '0xe1ebdf64f7f3a31723e767a561345f958233bb7d',
  'dollar_value': '-1431.5080078125',
  'transfers': [0, 1]}]

In [ ]:
import web3
from web3 import Web3
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
from web3.contract import Contract
...
contract = web3.eth.contract(address=Web3.toChecksumAddress(tokenAddress), abi=abi)
contract.functions.decode_function_input(transaction.input)

In [39]:
from web3 import Web3, EthereumTesterProvider
w3 = Web3(EthereumTesterProvider())
w3.is_connected()
#web3.eth.get_transaction(tx_hash)

True

In [180]:
# main.py
from web3 import Web3
import json 

# Setup
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/l-WPR6c5Gh6xVA8uOEuH1xy8VfocT0uH"
w3 = Web3(Web3.HTTPProvider(alchemy_url))

# Get the information of a transaction
tx = w3.eth.get_transaction('0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581')
print(tx)

# Get ABI for smart contract NOTE: Use "to" address as smart contract 'interacted with'
abi_endpoint = f"https://api.etherscan.io/api?module=contract&action=getabi&address={tx['to']}&apikey={eth_scan_api_key}"
abi = json.loads(requests.get(abi_endpoint).text)


# Create Web3 contract object
contract = w3.eth.contract(address=tx["to"], abi=abi["result"])


# Decode input data using Contract object's decode_function_input() method
func_obj, func_params = contract.decode_function_input(tx["input"])

AttributeDict({'blockHash': HexBytes('0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a2364ffa8cab9910143ee01'), 'blockNumber': 19412019, 'hash': HexBytes('0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581'), 'yParity': 0, 'accessList': [], 'transactionIndex': 2, 'type': 2, 'nonce': 1799, 'input': HexBytes('0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'), 'r': HexBytes('0xad8743a8c08357fb122fe099ce4ab1cc5155197bf17da09dd74b9bc435edf713'), 's': HexBytes('0x6ac3aca96164f1bf4582dc771aa945939eccac0c562a288a6c9

In [181]:
tx

AttributeDict({'blockHash': HexBytes('0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a2364ffa8cab9910143ee01'),
 'blockNumber': 19412019,
 'hash': HexBytes('0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581'),
 'yParity': 0,
 'accessList': [],
 'transactionIndex': 2,
 'type': 2,
 'nonce': 1799,
 'input': HexBytes('0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'),
 'r': HexBytes('0xad8743a8c08357fb122fe099ce4ab1cc5155197bf17da09dd74b9bc435edf713'),
 's': HexBytes('0x6ac3aca96164f1bf4582dc771aa945939eccac0c5

In [81]:
func_params

{'amountOut': 400000000000000000,
 'amountInMax': 872048734409341514734,
 'path': ['0xb0699d63aef20dF3F1cfFa9Ca2bB8670416271d2',
  '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],
 'to': '0x36A2fFB33C1B427C46C3D30Adac3cA4e8ED36179',
 'deadline': 1710163835}

In [82]:
from hexbytes import HexBytes
hb = HexBytes('0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
hb

HexBytes('0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')

In [85]:
# Create Web3 contract object
contract2 = w3.eth.contract(address=tx["to"], abi=abi["result"])
func_obj2, func_params2 = contract2.decode_function_input(hb)

In [96]:
tx_hash = '0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581'
info = w3.eth.get_transaction(tx_hash)
info

AttributeDict({'blockHash': HexBytes('0x2c1ca96b35f726a2171d3ebc050e390aca8dd1f83a2364ffa8cab9910143ee01'),
 'blockNumber': 19412019,
 'hash': HexBytes('0x1ee8d8e23e4a026ce8afbf2dc79c196c6b1d43d5e9f8ebb80b968a367ed5e581'),
 'yParity': 0,
 'accessList': [],
 'transactionIndex': 2,
 'type': 2,
 'nonce': 1799,
 'input': HexBytes('0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'),
 'r': HexBytes('0xad8743a8c08357fb122fe099ce4ab1cc5155197bf17da09dd74b9bc435edf713'),
 's': HexBytes('0x6ac3aca96164f1bf4582dc771aa945939eccac0c5

In [93]:
info['input']

HexBytes('0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')

In [94]:
func_obj2, a = contract2.decode_function_input(info['input'])

In [95]:
func_params2

{'amountOut': 400000000000000000,
 'amountInMax': 872048734409341514734,
 'path': ['0xb0699d63aef20dF3F1cfFa9Ca2bB8670416271d2',
  '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],
 'to': '0x36A2fFB33C1B427C46C3D30Adac3cA4e8ED36179',
 'deadline': 1710163835}

# calculate the difference (in the coin) between position we have vs. position 0

We have a difference of 42 BRETT

In [143]:
872048734409341514734 - 807452531860501402532

64596202548840112202

In [145]:
872048734409341514734 - 76454166803

872048734332887347931

In [154]:
872048734409341514734 - (872048734409341514734/0.41)*0.01009042093465994 

8.50586932444152e+20

In [170]:
76454166803*200373

15319350764817519

In [174]:
872048734409341514734 - 76454166803*200373*1000

856729383644523995734

In [176]:
0.000000076454166803*131980 # Gas Price times usage by txn 

0.01009042093465994

In [177]:
#gasPrice in wei in hexa * gas in wei in hex
gas = 200373 #no
gasPrice = 76454166803 #ok
gas*gasPrice

15319350764817519

In [179]:
#cumulativeGasUsed	0x6f884  456836
# effectiveGasPrice	0x11cd060113  76454166803
# gasUsed	0x2038c   131980


76454166803/1000000000000000000 * 131980

0.01009042093465994

# Calculate the cost in usd and eth of the difference 
use dune
there is no BRETT token in dune - asked felix
do the analysis again with the 3 transaction (transaction hash 0xee8fd2c76181afa14ca0da158e0a01bba2d3df8e62c5da122ac1d77912aab66e) because there is the DMT token in dune

Here we need to :
- get the price for the first token (here BRETT) (if possible) at that minute (check if there is any different value in the price dependent on the columns returned from dune)
- get the price for the second token (here ETH) (if possible) at that minute
- translate the difference in token 2 (here what is 42 BRETT in ETH)
- calculate the price difference for the tokens for which we have the price

Filter by contract address if the data exists in dune. Ask Felix if this is important information as it is not always in dune
